In [ ]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import cv2
import numpy as np
import os
import json

In [ ]:
# Percorsi input/output
PDF_PATH = "Informativa_Riepilogativa_dei_Servizi_di_Pagamento_30giu2025.pdf"
OUTPUT_JSON = "output_spese.json"

In [ ]:
# Percorsi input/output
PDF_PATH = "file.pfd"
OUTPUT_JSON = "output_spese.json"

def preprocess_image(image: Image.Image) -> Image.Image:
    # Converti PIL Image in OpenCV image per miglioramento
    cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    return Image.fromarray(thresh)

def extract_text_from_pdf(pdf_path: str):
    print(f"Converto il PDF in immagini: {pdf_path}")
    images = convert_from_path(pdf_path)
    full_text = ""

    for i, image in enumerate(images):
        print(f"Processo pagina {i + 1}")
        # Preprocessing base (facoltativo)
        processed_image = preprocess_image(image)

        text = pytesseract.image_to_string(processed_image, lang="ita+eng")
        full_text += f"--- Pagina {i + 1} ---\n{text}\n"

    return full_text

def parse_transactions(raw_text: str):
    # Funzione di parsing *molto basilare* da personalizzare secondo il formato dei tuoi estratti conto
    transactions = []
    lines = raw_text.split("\n")
    for line in lines:
        if any(char.isdigit() for char in line) and "€" in line:
            transactions.append({"riga": line.strip()})
    return transactions


In [ ]:
#def main():
    #raw_text = extract_text_from_pdf(PDF_PATH)
    #transactions = parse_transactions(raw_text)

    #print(f"Scrivo {len(transactions)} transazioni in {OUTPUT_JSON}")
    #with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    #    json.dump(transactions, f, indent=2, ensure_ascii=False)

In [ ]:
import pdfplumber
import pandas as pd

In [ ]:
# Apri il file PDF
with pdfplumber.open(PDF_PATH) as pdf:
    # Itera su ogni pagina
    testi = []
    tabelle = []
    for pagina in pdf.pages:
        # Estrai il testo dalla pagina
        testi.append(pagina.extract_text()) 
        
        # Prova a estrarre tabelle dalla pagina
        tabs = pagina.extract_table({
            "vertical_strategy": "lines",         # oppure "text"
            "horizontal_strategy": "lines",       # oppure "text"
            "intersection_tolerance": 5,          # aumenta se le linee non si incrociano perfettamente
            "snap_tolerance": 3,                  # tolleranza per 'agganciare' le linee ai testi
            "join_x_tolerance": 2,                # unisce linee verticali vicine
            "join_y_tolerance": 2,                # unisce linee orizzontali vicine
            "edge_min_length": 3,                 # ignora linee troppo corte
        })
        intestazione = tabs[0][0]
        for i, tabs in enumerate(tabs):
            print(f"\nTabella {i+1}:")
            tabelle.extend(tabs[1:])


In [ ]:
for ii, testo in enumerate(testi):
    print(f'=============== Testo {ii+1:02d} ===============')
    print(testo)

In [ ]:
trx_df = pd.DataFrame(tabelle,columns=intestazione)

In [ ]:
trx_df

In [ ]:
import pdfplumber.table


with pdfplumber.open(PDF_PATH) as pdf:
    page = pdf.pages[1]
    tab = page.extract_tables()[0]
    # Crea un oggetto immagine della pagina
    im = page.to_image(resolution=150)

    # Disegna le linee rilevate nella pagina
    #im.draw_lines(page.lines, stroke="red")
    im.debug_tablefinder({
            "vertical_strategy": "lines",         # oppure "text"
            "horizontal_strategy": "lines",       # oppure "text"
            "explicit_vertical_lines": [page.width*0.05, page.width*0.95],
            #"explicit_horizontal_lines": [20, 800]
            #"intersection_tolerance": 20,          # aumenta se le linee non si incrociano perfettamente
            "snap_x_tolerance": 9,                  # tolleranza per 'agganciare' le linee ai testi
            #"join_x_tolerance": 20,                # unisce linee verticali vicine
            #"join_y_tolerance": 2,                # unisce linee orizzontali vicine
            #"edge_min_length": 1,                 # ignora linee troppo corte
        })


    # Salva il risultato in un'immagine PNG
    #im.save("debug_tabella.png")
    

In [ ]:
im


In [ ]:
pd.DataFrame(columns=['a']).

In [ ]:
import pandas as pd
import pdfplumber

In [ ]:
trx_df = pd.DataFrame()
with pdfplumber.open(PDF_PATH) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables({
            "vertical_strategy": "lines",         # oppure "text"
            "horizontal_strategy": "lines",       # oppure "text"
            "explicit_vertical_lines": [page.width*0.05, page.width*0.95],
            "snap_x_tolerance": 9,                  # tolleranza per 'agganciare' le linee ai testi
        })
        for tab in tables:
            trx_df = pd.concat([trx_df, pd.DataFrame(tab[1:], columns=tab[0]).dropna(thresh=2)], ignore_index=True)

In [ ]:
trx_df.columns

In [ ]:
description_col = 'DESCRIZIONE OPERAZIONE'

In [ ]:
if trx_df.iloc[-1][description_col] == 'TOTALE':
    trx_df = trx_df.iloc[:-1]
#trx_df.dropna(thresh=2)

In [ ]:
trx_df.loc[16][description_col]

In [ ]:
import pdfplumber
from datetime import datetime
from data_models.transaction import OcrTransactionRaw

In [ ]:

def ReadDigitalPdf(file_path):

    ocr_transactions = []

    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables({
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines",
                "explicit_vertical_lines": [page.width * 0.05, page.width * 0.95],
                "snap_x_tolerance": 9,
            })
            for tab in tables:
                for row_idx, row in enumerate(tab, start=1):
                    #if row.count(None) + row.count('') >= len(row) - 2 or isinstance(row[1][0], ):
                    #    continue  # ignora righe quasi vuote

                    #row_dict = dict(zip(headers, row))
                    print(row)
                    
                    try:
                        # for amt in (row[3], row[4]):
                        #     if (amt is not None) and (amt != ''):
                        #         if ',' in amt:
                        #             amt = amt.replace(".", "").replace(",", ".")
                        #         amount += c*float(amt)
                        #     c *= -1
                        tx = OcrTransactionRaw(
                            filename=file_path,
                            row=row_idx,
                            booking_dt=row[0],#datetime.strptime(row[0], "%d/%m/%Y").date() if row[2] else None,
                            value_dt=row[1],#datetime.strptime(row[1], "%d/%m/%Y").date(),
                            amount=row[4],
                            negative_amount= row[3],
                            description=row[2],
                        )
                        ocr_transactions.append(tx)
                    except Exception as e:
                        print(f"Errore nel parsing della riga {row_idx}: {e}")
    return ocr_transactions

In [ ]:
from data_ingestion import ocr

In [ ]:
# Dovrei spostare il doppio importo dentro la classe rawocr?
PDF_PATH = "Informativa_Riepilogativa_dei_Servizi_di_Pagamento_30giu2025.pdf"

trxs = ocr.ReadDigitalPdf(PDF_PATH)

In [ ]:
trxs